In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import LdaModel, CoherenceModel, LdaMulticore
import gensim.corpora as corpora
import time
from gensim import models, corpora, similarities
import matplotlib.pyplot as plt
from scipy.stats import entropy
from scipy.spatial import distance
from tqdm.notebook import tqdm

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("dataset_nlp_clean.csv")
df['nlp_lemma'] = df['nlp_lemma'].replace(np.NaN, '')

In [6]:
corpus_list = []
tokens_list = []
for index, row in df.iterrows():
    texts = row['nlp_lemma']
    # corpus = [id2word.doc2bow(text) for text in texts]
    corpus_list.append(texts)
    tokens_list.append(texts.split())
df['tokens'] = tokens_list
df['corpus'] = corpus_list

In [7]:
df.head()

,Código del expediente,Número del procedimiento,Carácter del procedimiento,Forma del procedimiento,Artículo de excepción,REFERENCIA_EXPEDIENTE,Título del expediente,Plantilla del expediente,Descripción del Anuncio,Clave de la UC,Nombre de la UC,Operador,Correo electrónico,Entidad federativa,Tipo de contratación,Publicación del anuncio,Vigencia del anuncio,Clave COG,Fecha de creación,Fecha de última modificacion,Dirección del anuncio,nlp,nlp_lemma,tokens,corpus
0,50739,EO-823001999-N8-2011,Nacional,Presencial,NaN,NaN,Mejoramiento de la iluminación de la Iglesia San Miguel,V20150817 30. Licitación Pública Nacional de Obra Pública (Norma Estatal),Mejoramiento de la iluminación de la Iglesia San Miguel,823001999,QROO-Cozumel-Dirección de Obras Públicas #823001999,Gabriel Caamal Caamal,compranetobraspublicas@cozumel.gob.mx,Quintana Roo,Obra Pública,2020-08-10 12:49,2011-08-17 07:00,6170,2011/07/2910:40,2020/08/31 12:51,https://compranet.funcionpublica.gob.mx/esop/guest/go/opportunity/detail?opportunityId=29953,mejoramiento de la iluminacion de la iglesia san miguel,mejoramiento iluminacion iglesia santo miguel,"[mejoramiento, iluminacion, iglesia, santo, miguel]",mejoramiento iluminacion iglesia santo miguel
1,51475,EA-823001999-N10-2011,Nacional,Presencial,NaN,NaN,Adquisición de Uniformes para la dirección de Seguridad Pública y Transito,V20150817 28. Licitación Pública Nacional de Adquisiciones (Norma Estatal),Adquisición de Uniformes para la dirección de Seguridad Pública y Transito,823001999,QROO-Cozumel-Dirección de Obras Públicas #823001999,Gabriel Caamal Caamal,compranetobraspublicas@cozumel.gob.mx,Quintana Roo,Adquisiciones,2020-08-14 11:19,2011-08-17 12:00,2830,2011/08/0112:14,2020/08/18 11:32,https://compranet.funcionpublica.gob.mx/esop/guest/go/opportunity/detail?opportunityId=1878185,adquisicion de uniformes para la direccion de seguridad publica y transito,adquisicion uniforme direccion seguridad publicar transitar,"[adquisicion, uniforme, direccion, seguridad, publicar, transitar]",adquisicion uniforme direccion seguridad publicar transitar
2,61495,AO-811027990-N12-2011,NaN,NaN,Art. 43 s/f,DGOP/HABITAT/PROY/007-11,PROYECTO EJECUTIVO JARDIN VECINAL COLONIA LAS AMERICAS,06. Adjudicación Directa LOPSRM,PROYECTO EJECUTIVO JARDIN VECINAL COLONIA LAS AMERICAS DE ESTA CIUDAD DE SALAMANCA GTO.,811027990,GTO-Salamanca-DIRECCION GENERAL DE OBRAS PUBLICAS #811027990,Adriana Ortega González,adriana.ortega@salamanca.gob.mx,Guanajuato,Servicios Relacionados con la OP,2020-03-20 12:35,2011-08-23 23:00,6290,2011/08/2316:57,2020/03/24 14:26,https://compranet.funcionpublica.gob.mx/esop/guest/go/opportunity/detail?opportunityId=34849,proyecto ejecutivo jardin vecinal colonia las americas de esta ciudad de salamanca gto.,proyectar ejecutivo jardin vecinal colonia americas ciudad salamanca gto,"[proyectar, ejecutivo, jardin, vecinal, colonia, americas, ciudad, salamanca, gto]",proyectar ejecutivo jardin vecinal colonia americas ciudad salamanca gto
3,65885,IO-815060972-N7-2011,Nacional,Mixta,Art. 43 s/f,NR-OP-IR-PREP-15060-EMF-002/2011,CONSTRUCCION DE DEPORTIVO BICENTENARIO,V20151220 19. Invitación a cuando menos tres Nacional LOPSRM,CONSTRUCCION DE DEPORTIVO BICENTENARIO,815060972,MEX-Nicolás Romero-Direccion de Obras Publicas #815060972,Paulino Mata González,arqpau7@hotmail.com,México,Obra Pública,2020-06-08 10:26,2011-09-23 11:00,6120,2011/09/0109:55,2020/06/08 10:26,https://compranet.funcionpublica.gob.mx/esop/guest/go/opportunity/detail?opportunityId=1855084,construccion de deportivo bicentenario,construccion deportivo bicentenario,"[construccion, deportivo, bicentenario]",construccion deportivo bicentenario
4,65850,IO-815060972-N8-2011,Nacional,Mixta,Art. 43 s/f,NR-OP-IR-PREP-15060-EMF-001/2011,ACONDICIONAMIENTO DE CENTRO CULTURAL Y DEPORTIVO,V20151220 19. Invitación a cuando menos tres Nacional LOPSRM,ACONDICIONAMIENTO DE CENTRO CULTURAL Y DEPORTIVO,815060972,MEX-Nicolás Romero-Direccion de Obras Publicas #815060972,Paulino Mata Go

In [ ]:
# id2word = corpora.Dictionary(df['texts'])
# corpus = [id2word.doc2bow(text) for text in df['texts']]

In [8]:
def train_lda(data, alfa, beta, num_topics=10):
    """
    This function trains the lda model
    We setup parameters like number of topics, the chunksize to use in Hoffman method
    We also do 2 passes of the data since this is a small dataset, so we want the distributions to stabilize
    """
    dictionary = corpora.Dictionary(data['tokens'])
    corpus = [dictionary.doc2bow(doc) for doc in data['tokens']]
    t1 = time.time()
    # low alpha means each document is only represented by a small number of topics, and vice versa
    # low eta means each topic is only represented by a small number of words, and vice versa
    # lda = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, alpha=alfa, eta=beta, minimum_probability=0.0, passes=10, random_state=0)
                   
    lda = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, alpha=alfa, eta=beta, minimum_probability=0.0, passes=10, chunksize=300, random_state=0, workers=None)
    coherence_model_lda = CoherenceModel(model=lda, texts=data['tokens'].tolist(), dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    t2 = time.time()
    print("Time to train LDA model on ", len(df), "articles: ", (t2-t1)/60, "min")
    return dictionary,corpus,lda,coherence_lda 

In [14]:
kas = range(2, 11, 1)

# Alpha parameter
alpha = list(np.arange(0.01, 1.0, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1.0, 0.3))
beta.append('symmetric')

best_i = 0
best_coherence = 0

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
with tqdm(total = len(kas), bar_format='{bar}|{desc}{percentage:3.0f}% {r_bar}') as pbar:
    for i in kas:
        for a in alpha:
            for b in beta:
                dictionary,corpus,lda,coherence = train_lda(df, alfa=a, beta=b, num_topics=i)
                print(f"Coherence for {i} topics (Alpha={a}, Beta={b}): {coherence}")
                if coherence > best_coherence:
                    best_i = i
                    best_coherence = coherence
                    best_alfa = a
                    best_beta = b
                    
                model_results['Topics'].append(i)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(coherence)
        pbar.update(1)
            
model_results_df = pd.DataFrame(model_results)
                
print(f"Best num of topics: {best_i} (Alpha={best_alfa}, Beta={best_beta}) with coherence = {best_coherence}")

n LDA model on  96122 articles:  1.0908902684847515 min
Coherence for 6 topics (Alpha=0.61, Beta=0.31): 0.5991389789325067
Time to train LDA model on  96122 articles:  1.1061764081319174 min
Coherence for 6 topics (Alpha=0.61, Beta=0.61): 0.5961713532853271
Time to train LDA model on  96122 articles:  1.123843002319336 min
Coherence for 6 topics (Alpha=0.61, Beta=0.9099999999999999): 0.6099456776260759
Time to train LDA model on  96122 articles:  1.0785930315653482 min
Coherence for 6 topics (Alpha=0.61, Beta=symmetric): 0.6048927746638019
Time to train LDA model on  96122 articles:  1.0142196337382 min
Coherence for 6 topics (Alpha=0.9099999999999999, Beta=0.01): 0.6324031755842584
Time to train LDA model on  96122 articles:  1.0334973891576131 min
Coherence for 6 topics (Alpha=0.9099999999999999, Beta=0.31): 0.6379672949826902
Time to train LDA model on  96122 articles:  1.0542896469434102 min
Coherence for 6 topics (Alpha=0.9099999999999999, Beta=0.61): 0.607419843980743
Time to tra

In [9]:
dictionary,corpus,lda,coherence = train_lda(df, alfa=0.9099999999999999, beta='symmetric', num_topics=5)

Time to train LDA model on  96122 articles:  1.3706595460573832 min


In [10]:
# show_topics method shows the the top num_words contributing to num_topics number of random topics
lda.show_topics(num_topics=5, num_words=20)

[(0,
  '0.054*"servicio" + 0.023*"profesional" + 0.016*"apoyar" + 0.013*"nacional" + 0.012*"elaboracion" + 0.009*"e" + 0.009*"informacion" + 0.007*"publicar" + 0.007*"control" + 0.007*"seguimiento" + 0.006*"actividad" + 0.006*"coadyuvar" + 0.006*"sistema" + 0.006*"administrativo" + 0.005*"atencion" + 0.005*"produccion" + 0.005*"programar" + 0.005*"analisis" + 0.004*"instituto" + 0.004*"administracion"'),
 (1,
  '0.059*"adquisicion" + 0.056*"material" + 0.026*"curacion" + 0.019*"medicamento" + 0.014*"laboratorio" + 0.011*"accesorio" + 0.011*"adjudicacion" + 0.011*"directo" + 0.010*"grupo" + 0.009*"suministro" + 0.008*"envasar" + 0.008*"medicos" + 0.007*"miligramo" + 0.006*"alto" + 0.006*"contener" + 0.006*"vegetal" + 0.006*"equipar" + 0.006*"mililitro" + 0.006*"covid-19" + 0.006*"suministrar"'),
 (2,
  '0.040*"rural" + 0.038*"establecer" + 0.037*"programar" + 0.036*"desarrollar" + 0.028*"integral" + 0.019*"direccion" + 0.016*"ley" + 0.016*"linear" + 0.015*"tecnica" + 0.015*"presentar" +

In [16]:
lda.
for i in range(5):
    print(f"Showing topic: {i}")
    print(f"\t{lda.show_topic(topicid=i, topn=20)}")
    print("\n")

Showing topic: 0
	[('servicio', 0.05419069), ('profesional', 0.023073554), ('apoyar', 0.015945783), ('nacional', 0.013459353), ('elaboracion', 0.012354963), ('e', 0.009488567), ('informacion', 0.009044206), ('publicar', 0.007168093), ('control', 0.006739752), ('seguimiento', 0.0066884444), ('actividad', 0.006308964), ('coadyuvar', 0.0058020013), ('sistema', 0.0056813005), ('administrativo', 0.005599497), ('atencion', 0.0053372723), ('produccion', 0.0050806287), ('programar', 0.005032489), ('analisis', 0.0045469594), ('instituto', 0.004474118), ('administracion', 0.0043973215)]


Showing topic: 1
	[('adquisicion', 0.058639824), ('material', 0.055856295), ('curacion', 0.02592756), ('medicamento', 0.019294241), ('laboratorio', 0.0144603755), ('accesorio', 0.01148835), ('adjudicacion', 0.0114041725), ('directo', 0.010892282), ('grupo', 0.009802117), ('suministro', 0.008662567), ('envasar', 0.008417626), ('medicos', 0.0075705573), ('miligramo', 0.006756567), ('alto', 0.0064423466), ('conten

In [17]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num) + 1, round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda, corpus=corpus, texts=df['corpus'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

df_dominant_topic['expediente'] = df['Código del expediente']
df_dominant_topic['corpus'] = df['corpus']

# Show
df_dominant_topic.head(10)
df_dominant_topic.to_csv("dataset_lda.csv", encoding='utf8', index=False)

In [ ]:
# select and article at random from train_df
random_article_index = np.random.randint(len(df))
bow = dictionary.doc2bow(df.iloc[random_article_index,16])
print(random_article_index)

In [ ]:
print(df.iloc[random_article_index,0])

In [ ]:
# get the topic contributions for the document chosen at random above
doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=bow)])

In [ ]:
# bar plot of topic distribution for this document
fig, ax = plt.subplots(figsize=(12,6));
# the histogram of the data
patches = ax.bar(np.arange(len(doc_distribution)), doc_distribution)
ax.set_xlabel('Topic ID', fontsize=15)
ax.set_ylabel('Topic Contribution', fontsize=15)
ax.set_title("Topic Distribution for Article " + str(random_article_index), fontsize=20)
ax.set_xticks(np.linspace(10,5,1))
fig.tight_layout()
plt.show()

In [ ]:
# print the top 5 contributing topics and their words
for i in doc_distribution.argsort()[-5:][::-1]:
    print(i, lda.show_topic(topicid=i, topn=10), "\n")

Similarity Queries on Unseen Data

In [ ]:
# select and article at random from test_df
random_article_index = np.random.randint(len(df))
print(random_article_index)

In [ ]:
new_bow = dictionary.doc2bow(df.iloc[random_article_index,16])

In [ ]:
print(df.iloc[random_article_index,0])

In [ ]:
new_doc_distribution = np.array([tup[1] for tup in lda.get_document_topics(bow=new_bow)])

In [ ]:
new_doc_distribution

In [ ]:
# bar plot of topic distribution for this document
fig, ax = plt.subplots(figsize=(12,6));
# the histogram of the data
patches = ax.bar(np.arange(len(new_doc_distribution)), new_doc_distribution)
ax.set_xlabel('Topic ID', fontsize=15)
ax.set_ylabel('Topic Contribution', fontsize=15)
ax.set_title("Topic Distribution for an Unseen Article", fontsize=20)
ax.set_xticks(np.linspace(10,5,1))
fig.tight_layout()
plt.show()

In [ ]:
# print the top 8 contributing topics and their words
for i in new_doc_distribution.argsort()[-5:][::-1]:
    print(i, lda.show_topic(topicid=i, topn=10), "\n")

Similarity Query

In [ ]:
# we need to use nested list comprehension here
# this may take 1-2 minutes...
doc_topic_dist = np.array([[tup[1] for tup in lst] for lst in lda[corpus]])
doc_topic_dist.shape

In [ ]:
doc_topic_dist


braycurtis(u, v[, w])

Compute the Bray-Curtis distance between two 1-D arrays.

canberra(u, v[, w])

Compute the Canberra distance between two 1-D arrays.

chebyshev(u, v[, w])

Compute the Chebyshev distance.

cityblock(u, v[, w])

Compute the City Block (Manhattan) distance.

correlation(u, v[, w, centered])

Compute the correlation distance between two 1-D arrays.

cosine(u, v[, w])

Compute the Cosine distance between 1-D arrays.

euclidean(u, v[, w])

Computes the Euclidean distance between two 1-D arrays.

jensenshannon(p, q[, base])

Compute the Jensen-Shannon distance (metric) between two 1-D probability arrays.

mahalanobis(u, v, VI)

Compute the Mahalanobis distance between two 1-D arrays.

minkowski(u, v[, p, w])

Compute the Minkowski distance between two 1-D arrays.

seuclidean(u, v, V)

Return the standardized Euclidean distance between two 1-D arrays.

sqeuclidean(u, v[, w])

Compute the squared Euclidean distance between two 1-D arrays.

wminkowski(u, v, p, w)

Compute the weighted Minkowski distance between two 1-D arrays.

In [ ]:
def calcula_distancia(query, matrix, metrica):
    """
    This function implements a Jensen-Shannon similarity
    between the input query (an LDA topic distribution for a document)
    and the entire corpus of topic distributions.
    It returns an array of length M where M is the number of documents in the corpus
    """
    
    sim = [metrica(data,query) for data in matrix]
    sim = np.array(sim, dtype=np.float64)
    
    return sim

In [ ]:
def get_most_similar_documents(query,matrix,k=15, metrica=distance.cosine):
    """
    This function implements the Jensen-Shannon distance above
    and retruns the top k indices of the smallest jensen shannon distances
    """
    sims = calcula_distancia(query,matrix, metrica) # list of jensen shannon distances
    sims = np.array(sims, dtype=np.float32)
    return sims.argsort()[:k] # the top k positional index of the smallest Jensen Shannon distances

In [ ]:
# this is surprisingly fast
metrica=distance.jensenshannon
most_sim_ids = get_most_similar_documents(new_doc_distribution,doc_topic_dist, metrica=metrica)
most_similar_df = df.iloc[most_sim_ids]
most_similar_df['titulo_caso']

In [ ]:
# this is surprisingly fast
metrica=distance.euclidean
most_sim_ids = get_most_similar_documents(new_doc_distribution,doc_topic_dist, metrica=metrica)
most_similar_df = df.iloc[most_sim_ids]
most_similar_df['titulo_caso']

In [ ]:
# this is surprisingly fast
metrica=distance.cosine
most_sim_ids = get_most_similar_documents(new_doc_distribution,doc_topic_dist)
most_similar_df = df.iloc[most_sim_ids]
most_similar_df['titulo_caso']

In [ ]:
# this is surprisingly fast
metrica=distance.chebyshev
most_sim_ids = get_most_similar_documents(new_doc_distribution,doc_topic_dist)
most_similar_df = df.iloc[most_sim_ids]
most_similar_df['titulo_caso']

In [18]:
import pyLDAvis.gensim
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(vis, "../../../data/lda/lda.html")
pyLDAvis.display(vis)